# Detecting an STR mutation in the *APC* gene

Before working through this notebook, you should have completed the steps outlined in this repo's `README.md` file. i.e., you should have run GangSTR to generate a VCF file of STR genotypes for our sequencing sample.  

In this notebook, we will programatically seach the VCF file to detect a STR mutation. Subsequently, we will try to interpret the possible effects of this mutation using the integrative genomics viewer (IGV) web app, Biopython, and Ensembl's variant effect predictor (VEP) tool.

First, let's load some libraries:

In [1]:
import numpy as np
import pandas as pd
from cyvcf2 import VCF
from Bio import SeqIO, Seq

Our first task consists of searching for previously generated VCF file for the mutated STR locus. There should only be one mutation. There are many ways to approach this. I will outline two options here, feel free to pick the one that appeals most to you (or try both, or something completely different...). 

One option is a "vanilla" Python file parsing approach: open the VCF file, step through it line by line, and extract the VCF entry of interest. 
Some useful functions for this approach are:  

```python
open()
str.startswith()
str.strip()
str.split()
```

Another approach is to use a dedicated VCF file parser that someone else developed. One example of such a library is [cyvcf2](https://brentp.github.io/cyvcf2/). Using the documentation for this library, you should be able to parse the VCF file and extract the entry containing the STR mutation.  

*Note: In general, it is a good idea to use existing libraries for things like file parsing unless you have a very, very good reason not to. Existing libraries that are used by many people are typically very robust. Different people will have used them in many different setting, so many mistakes and bugs will have been noticed and fixed already.*

In [170]:
# Your code goes here




## Inspecting the mutation in the IGV web app

Open the [IGV web app](https://igv.org/app/) in your browser of choice. It just look like this:

![](images/IGV_empty_example.png)

By default, the human reference genome 'GRCh38' is loaded in the viewer. However, we want to use our custom APC reference sequence. Click on the `Genome` button in the top left, and select `Local File...`. You will then be prompted to select a file to upload. Navigate to the `data/alignments/` folder, and select `APC.fa` **AND** `APC.fa.fai` and the same time, and press `Open`, like so:
![](images/IGV_upload_reference.png)

The IGV web app should now display the reference sequence. You can now inspect the nucleotide sequence if you zoom in a bit, although this is not very interesting by itself:  
![](images/IGV_just_APC.png)

Let's add some of the other information we have available! Press the `Tracks` button at the top, choose `Local File...` again, and &mdash; similarly to how you uploaded the reference genome &mdash; add our sequence alignment (`APC_mut.bam`) + index (`APC_mut.bam.bai`) to the viewer. Then, do the same for the VCF file generated by GangSTR (this file does not need an index). Finally, upload the two files in `data/genome_annotation` (ending in `.gtf.gz` and `.gtf.gz.tbi`). This pair of files is are in [GTF](https://www.ensembl.org/info/website/upload/gff.html) format. This is a file format for annotation genomic sequences. This particular GTF file tells us where the transcript, exons, and coding sequences of the *APC* gene are located. After uploading all information, the final result should look something like what is shown below (once you zoom in a bit). Take a moment to look at the different tracks: do you know what each track represents?
![](images/IGV_complete_example.png)

Now that all our information is loaded into the genome viewer, it's time to take a look at the mutation in our sample. Navigate to the location of the STR mutation using the coordinates that you got from the GangSTR VCF file in the first part of this notebook (go to `Help`, `Documentation` in the IGV web app and look under `Navigation` if you're having trouble).  

In what type of sequence does the STR mutation occur? Do you expect this mutation to have a functional impact on the protein transcribed from this gene? (especially pay attention to the genome annotation track, labelled `APC_canonical_relative_coordinates.gtf.gz`)

## Manual assessment of STR mutation impact

Detecting the longest possible ORF in wild type and mutation sequence

In [153]:
(141625 - 135054) 
135054 + divmod(141625 - 135054, 3)[0] * 3
(141625 - 135054) % 3

1

In [155]:
135054 + divmod(141625 - 135054, 3)[0] * 3

141624

In [158]:
def longest_orf(sequence: Seq, reverse_complement=False) -> Seq:
    max_length = 0
    for frame in (0, 1, 2):
        frame_translation = sequence[frame:].translate()
        stop_codon_idx = frame_translation.find("*")
        if stop_codon_idx == -1:
            current_frame_length = len(frame_translation)
        else:
            current_frame_length = stop_codon_idx

        if current_frame_length > max_length:
            max_length = current_frame_length
    return max_length

In [35]:
mut = SeqIO.read("../../results/APC_mut_consensus.fa", "fasta")
wt = SeqIO.read("../../data/reference/APC.fa", "fasta")


In [159]:
longest_orf(wt.seq[135054:141625])

2190

In [160]:
longest_orf(mut.seq[135054:141625])

817

In [103]:
# chr5:135,055-141,625
wt.seq[135055:141625].translate().find("*")

-1

In [94]:
mut.seq[135056:141625].translate().find("*")

2

In [87]:
print(wt.seq[135055:141628].translate().find("*"))
len(wt.seq[135055:141628].translate())

19


2191

In [102]:
wt.seq[135055:141628].translate()

Seq('QILRENNCLQTLLQHLKSHSLTIVSNACGTLWNLSARNPKDQEALWDMGAVSML...SV*')

In [62]:
print(mut.seq[135055:141624].translate().find("*"))
len(mut.seq[135055:141624].translate())

817


2189

In [63]:
mut.seq[135055:141628].translate()

Seq('QILRENNCLQTLLQHLKSHSLTIVSNACGTLWNLSARNPKDQEALWDMGAVSML...FKR')

In [75]:
wt.seq[137461:137501]

Seq('AAAGCACCTACTGCTGAAAAGAGAGAGAGTGGACCTAAGC')

In [77]:
wt.seq[137462:137501].translate()

Seq('KHLLLKRERVDLS')

In [78]:
mut.seq[137462:137501].translate()

Seq('KHLLLKREWT*AS')